In [1]:
import torch
from torch import nn


In [2]:
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0] -h+1), (X.shape[1] -w +1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            mul = (X[i:i+h, j:j+w] * K)
#             print(mul)
            Y[i,j] = mul.sum()
    
    return Y

In [3]:
X = torch.tensor([[1.0,2.0,3.0], [2.0,3.0,4.0],[3.0,4.0,5.0]])
K = torch.tensor([[1.0,2.0], [2.0,3.0]])

corr2d(X,K)

tensor([[18., 26.],
        [26., 34.]])

In [4]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [5]:
net = Conv2d(K.shape)

In [6]:
net(X)

tensor([[-4.1519, -5.4360],
        [-5.4360, -6.7201]], grad_fn=<AddBackward0>)

### Edge detection

In [7]:
X = torch.ones((8,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
K = torch.tensor([[1.0,-1.0]])

In [9]:
corr2d(X,K)
# detects horizontal edges

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [10]:
corr2d(X.t(),K)
# but not vertical edges

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [11]:
X = torch.ones((8,8))

X[:,2:3] = 0
X[:,4:6] = 0

print(X)
Y = corr2d(X,K)
Y, Y.shape
# interesting

tensor([[1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1.]])


(tensor([[ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0.,  1., -1.,  1.,  0., -1.,  0.]]),
 torch.Size([8, 7]))

In [12]:
K.shape

torch.Size([1, 2])

### Learning a convolution

In [15]:
X = torch.ones((6,8))
X[:,2:6] = 0

Y =  corr2d(X,K)

In [19]:
conv2d = nn.Conv2d(1,1,kernel_size=(1,2), bias=False)

X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
lr = 3e-2

print(X.shape)
print(Y.shape)

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    
    if (i+1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum()}')


torch.Size([1, 1, 6, 8])
torch.Size([1, 1, 6, 7])
batch 2, loss 14.947685241699219
batch 4, loss 4.777310848236084
batch 6, loss 1.7310880422592163
batch 8, loss 0.6711881756782532
batch 10, loss 0.2685655653476715


In [20]:
conv2d.weight.data

tensor([[[[ 1.0408, -0.9347]]]])

### Exercises

1. Construct an image X with diagonal edges.
    1. What happens if you apply the kernel K in this section to it?
        * zero matrix.
    2. What happens if you transpose X?
        * No change
    3. What happens if you transpose K?
        * zero matrix.
        
2. When you try to automatically find the gradient for the Conv2D class we created, what kind
of error message do you see?
    * I am able to do `net.weights.grad`, when I try `net.grad` I get the error `'Conv2d' object has no attribute 'grad'`

3. How do you represent a cross-correlation operation as a matrix multiplication by changing
the input and kernel tensors?
    * cross correlation is basically matrix multiplication between slices of tensorfrom X of the shape of kernel and summing.
    * It can be done by padding Kand X based on what is needed to multiply

4. Design some kernels manually.
    1. What is the form of a kernel for the second derivative?
        * okay in order to compute one way would be to manually compute the second derivative and then let see a kernel be made using backpropogation
    2. What is the kernel for an integral?
        * how do you actually make it manually
3. What is the minimum size of a kernel to obtain a derivative of degree d
        * dont know.

In [24]:
#1

X =  torch.zeros((8,8))
for i in range(X.shape[0]):
    X[i][i] = 1
X

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [39]:
K = torch.tensor([[1.0,-1.0]])
K.shape

torch.Size([1, 2])

In [40]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (Y[i:i+h, j : j +w] * K).sum()
    return Y

In [41]:
corr2d(X, K)

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [43]:
X.t(), X

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.]]))

In [44]:
corr2d(X, K.t())

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [47]:
Y = corr2d(X, K)
Y.shape

torch.Size([8, 7])

In [69]:
net = nn.Conv2d(1,1,K.shape, bias=False)

X = X.reshape(1,1,8,8)
Y = Y.reshape(1,1,8,7)

print(Y, X)
lr = 3e-2

for i in range(10):
    y_hat = net(X)
#     print(y_hat.shape)
    l = (y_hat - Y)**2
    
    net.zero_grad()
    
    l.sum().backward()
    
    net.weight.data -= lr * net.weight.grad
    
    print(f"for epoch {i} loss: {l.sum()}")


tensor([[[[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.]]]]) tensor([[[[1., 0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1.]]]])
for epoch 0 loss: 1.8485668897628784
for epoch 1 loss: 0.6218580007553101
for epoch 2 loss: 0.20919303596019745
for epoch 3 loss: 0.07037252932786942
for epoch 4 loss: 0.023673322051763535
for epoch 5 loss: 0.007963704876601696
for epoch 6 loss: 0.0026789901312440634
for epoch 7 loss: 0.0009012123919092119
for epoch 8 lo

In [70]:
print(net.weight.data[0][0])
corr2d(X, net.weight.data[0][0])

tensor([[-0.0019,  0.0012]])


tensor([], size=(1, 0))

In [73]:
try:
    net.grad
except Exception as e:
    print(e)

'Conv2d' object has no attribute 'grad'
